In [1]:
import json


def init():
    f = open('./CAIL_data/law.txt', 'r', encoding = 'utf8')
    law = {}
    lawname = {}
    line = f.readline()
    while line:
        lawname[len(law)] = line.strip()
        law[line.strip()] = len(law)
        line = f.readline()
    f.close()


    f = open('./CAIL_data/accu.txt', 'r', encoding = 'utf8')
    accu = {}
    accuname = {}
    line = f.readline()
    while line:
        accuname[len(accu)] = line.strip()
        accu[line.strip()] = len(accu)
        line = f.readline()
    f.close()


    return law, accu, lawname, accuname


law, accu, lawname, accuname = init()

In [2]:
def getClassNum(kind):
    global law
    global accu

    if kind == 'law':
        return len(law)
    if kind == 'accu':
        return len(accu)

In [3]:
def getName(index, kind):
    global lawname
    global accuname
    if kind == 'law':
        return lawname[index]

    if kind == 'accu':
        return accuname[index]


def gettime(time):
    #将刑期用分类模型来做
    v = int(time['imprisonment'])

    if time['death_penalty']:
        return 0
    if time['life_imprisonment']:
        return 1
    elif v > 10 * 12:
        return 2
    elif v > 7 * 12:
        return 3
    elif v > 5 * 12:
        return 4
    elif v > 3 * 12:
        return 5
    elif v > 2 * 12:
        return 6
    elif v > 1 * 12:
        return 7
    else:
        return 8


def getlabel(d, kind):
    global law
    global accu    
    # 做单标签
    if kind == 'law':
    # 返回多个类的第一个
#         return law[str(d['meta']['relevant_articles'])]
        return str(d['meta']['relevant_articles'])
    if kind == 'accu':
#         return accu[d['meta']['accusation']]
        return d['meta']['accusation']
    if kind == 'time':
#         return gettime(d['meta']['term_of_imprisonment'])
        return gettime(d['meta']['term_of_imprisonment'])
    
    return label
    


In [4]:
def read_trainData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    alltext = []
    
    accu_label = []
    law_label = []
    time_label = []

    line = fin.readline()
    while line:
        d = json.loads(line)
        alltext.append(d['fact'])
        accu_label.append(getlabel(d, 'accu'))
        law_label.append(getlabel(d, 'law'))
        time_label.append(getlabel(d, 'time'))
        line = fin.readline()
    fin.close()

    return alltext, accu_label, law_label, time_label

In [6]:
alltext, accu_label, law_label, time_label = read_trainData('./cail2018_big/cail2018_big.json')

In [9]:
fact = []
content = alltext
import jieba
for text in content:
    fact.append(' '.join(jieba.cut(text)))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.019 seconds.
Prefix dict has been built succesfully.


In [13]:
len(fact), len(accu_label), len(law_label), len(time_label)

(1710856, 1710856, 1710856, 1710856)

In [20]:
import pickle

pickle.dump(fact, open('facts.pkl', 'wb'))
facts2 = pickle.load(open('facts.pkl', 'rb'))

pickle.dump(accu_label, open('accu_label.pkl', 'wb'))
accu_label2 = pickle.load(open('accu_label.pkl', 'rb'))

pickle.dump(law_label, open('law_label.pkl', 'wb'))
law_label2 = pickle.load(open('law_label.pkl', 'rb'))

pickle.dump(accu_label, open('time_label.pkl', 'wb'))
time_label2 = pickle.load(open('time_label.pkl', 'rb'))

In [50]:
import re

law_label3 = [re.sub('[\[\]\' ]+', '',law_label2[i]).split(',') for i in range(len(law_label2))]


In [57]:
pickle.dump(law_label3, open('law_label.pkl', 'wb'))
law_label4 = pickle.load(open('law_label.pkl', 'rb'))

In [62]:
len(facts2), len(accu_label2), len(law_label4), len(time_label2)

(1710856, 1710856, 1710856, 1710856)